# Alternative sampling backends

In Bambi, the sampler used is automatically selected given the type of variables used in the model. For inference, Bambi supports both MCMC and variational inference. By default, Bambi uses PyMC's implementation of the adaptive Hamiltonian Monte Carlo (HMC) algorithm for sampling. Also known as the No-U-Turn Sampler (NUTS). This sampler is a good choice for many models. However, it is not the only sampling method, nor is PyMC the only library implementing NUTS. 

To this extent, Bambi supports multiple backends for MCMC sampling such as NumPyro and Blackjax. This notebook will cover how to use such alternatives in Bambi.

_Note_: Bambi utilizes [bayeux](https://github.com/jax-ml/bayeux) to access a variety of sampling backends. Thus, you will need to install the optional dependencies in the Bambi [pyproject.toml](https://github.com/bambinos/bambi/blob/main/pyproject.toml) file to use these backends.

In [4]:
import arviz as az
import bambi as bmb
import bayeux as bx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## bayeux

Bambi leverages `bayeux` to access different sampling backends. In short, `bayeux` lets you write a probabilistic model in JAX and immediately have access to state-of-the-art inference methods. 

Since the underlying Bambi model is a PyMC model, this PyMC model can be "given" to `bayeux`. Then, we can choose from a variety of MCMC methods to perform inference. Below, the list of alternative MCMC methods to use in Bambi is shown.

In [5]:
# Tensorflow probability based methods are currently not supported
mcmc_methods = [getattr(bx.mcmc, k).name for k in bx.mcmc.__all__ if "tfp" not in getattr(bx.mcmc, k).name ]
mcmc_methods

['blackjax_hmc',
 'blackjax_chees_hmc',
 'blackjax_meads_hmc',
 'blackjax_nuts',
 'blackjax_hmc_pathfinder',
 'blackjax_nuts_pathfinder',
 'flowmc_rqspline_hmc',
 'flowmc_rqspline_mala',
 'flowmc_realnvp_hmc',
 'flowmc_realnvp_mala',
 'numpyro_hmc',
 'numpyro_nuts']

`bayeux` lets us have access to Blackjax, FlowMC, and NumPyro backends. In the section below, we will show how to use these backends in Bambi.

## Specifying an `inference_method`

First, we simulate some data to use in the examples.

In [6]:
num_samples = 100
num_features = 1
noise_std = 1.0
random_seed = 42

np.random.seed(random_seed)

coefficients = np.random.randn(num_features)
X = np.random.randn(num_samples, num_features)
error = np.random.normal(scale=noise_std, size=num_samples)
y = X @ coefficients + error

data = pd.DataFrame({"y": y, "x": X.flatten()})

In [7]:
model = bmb.Model("y ~ x", data)

To use a different backend, we pass the name of the `bayeux` MCMC inference method to the `inference_method` parameter of the `fit` method.

### Blackjax

In [8]:
blackjax_nuts_idata = model.fit(inference_method="blackjax_nuts")
blackjax_nuts_idata

Inference data with groups:
	> posterior
	> sample_stats

Different backends have different naming conventions for the parameters specific to that MCMC method. Thus, to specify backend-specific parameters, pass your own `kwargs` to the `fit` method.

Each algorithm has a `.get_kwargs()` method that tells you how it will be called, and what functions are being called.

In [9]:
bx.Model.from_pymc(model.backend.model).mcmc.blackjax_nuts.get_kwargs()

{<function blackjax.adaptation.window_adaptation.window_adaptation(algorithm: Union[blackjax.mcmc.hmc.hmc, blackjax.mcmc.nuts.nuts], logdensity_fn: Callable, is_mass_matrix_diagonal: bool = True, initial_step_size: float = 1.0, target_acceptance_rate: float = 0.8, progress_bar: bool = False, **extra_parameters) -> blackjax.base.AdaptationAlgorithm>: {'logdensity_fn': <function bayeux._src.shared.constrain.<locals>.wrap_log_density.<locals>.wrapped(args)>,
  'is_mass_matrix_diagonal': True,
  'initial_step_size': 1.0,
  'target_acceptance_rate': 0.8,
  'progress_bar': False,
  'algorithm': blackjax.mcmc.nuts.nuts},
 'adapt.run': {'num_steps': 500},
 blackjax.mcmc.nuts.nuts: {'max_num_doublings': 10,
  'divergence_threshold': 1000,
  'integrator': <function blackjax.mcmc.integrators.generate_euclidean_integrator.<locals>.euclidean_integrator(logdensity_fn: Callable, kinetic_energy_fn: blackjax.mcmc.metrics.KineticEnergy) -> Callable[[blackjax.mcmc.integrators.IntegratorState, float], bla

Now, we can identify the kwargs we would like to change and pass to the `fit` method.

In [10]:
kwargs = {
        "adapt.run": {"num_steps": 500},
        "num_chains": 4,
        "num_draws": 250,
        "num_adapt_draws": 250
}

blackjax_nuts_idata = model.fit(inference_method="blackjax_nuts", **kwargs)
blackjax_nuts_idata

Inference data with groups:
	> posterior
	> sample_stats

### NumPyro

In [11]:
numpyro_nuts_idata = model.fit(inference_method="numpyro_nuts")
numpyro_nuts_idata

sample: 100%|██████████| 1500/1500 [00:02<00:00, 551.76it/s]


Inference data with groups:
	> posterior
	> sample_stats

### flowMC

In [12]:
flowmc_idata = model.fit(inference_method="flowmc_realnvp_hmc")
flowmc_idata

No autotune found, use input sampler_params
Training normalizing flow


Tuning global sampler: 100%|██████████| 5/5 [00:51<00:00, 10.23s/it]


Starting Production run


Production run: 100%|██████████| 5/5 [00:00<00:00,  9.38it/s]


Inference data with groups:
	> posterior

## Sampler comparisons

With ArviZ, we can compare the inference result summaries of the samplers. _Note:_ We can't use `az.compare` as not each inference data object returns the pointwise log-probabilities. Thus, an error would be raised.

In [16]:
az.summary(blackjax_nuts_idata)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
y_sigma,0.945,0.070,0.819,1.080,0.002,0.002,1044.0,667.0,1.0
Intercept,0.018,0.089,-0.156,0.185,0.003,0.002,844.0,733.0,1.0
x,0.358,0.105,0.163,0.554,0.004,0.003,829.0,767.0,1.0


In [17]:
az.summary(numpyro_nuts_idata)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
Intercept,0.022,0.097,-0.149,0.217,0.001,0.001,7412.0,5758.0,1.0
x,0.359,0.105,0.159,0.555,0.001,0.001,7406.0,5967.0,1.0
y_sigma,0.947,0.069,0.822,1.079,0.001,0.001,7371.0,5405.0,1.0


In [18]:
az.summary(flowmc_idata)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
y_sigma,0.946,0.067,0.825,1.076,0.001,0.001,6260.0,5213.0,1.00
Intercept,0.013,0.093,-0.165,0.190,0.003,0.002,924.0,1302.0,1.02
x,0.359,0.103,0.166,0.556,0.001,0.001,5132.0,5790.0,1.00


## Summary

Thanks to `bayeux`, we can use three different sampling backends and 10+ alternative MCMC methods in Bambi. Using these methods is as simple as passing the inference name to the `inference_method` of the `fit` method.

In [20]:
%load_ext watermark
%watermark -n -u -v -iv -w

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Last updated: Fri Mar 01 2024

Python implementation: CPython
Python version       : 3.11.7
IPython version      : 8.21.0

arviz     : 0.17.0
bambi     : 0.13.1.dev16+g9a1387a7.d20240204
bayeux    : 0.1.9
numpy     : 1.26.3
matplotlib: 3.8.2
pandas    : 2.2.0

Watermark: 2.4.3

